## Notebook Setup

### Imports
* **Run this cell immediately after opening the notebook by placing the cursor in the cell and hitting `SHIFT + ENTER`.**
* Otherwise, leave this cell alone

In [3]:
# Imports for telling Python where to find IRScout Python modules
import sys
import os
import re
sys.path.append('..\Server')

# Set Current working directory to irsScouting2017\Server
ptn = re.compile("jupyter$")
if ptn.search(os.getcwd()) is not None:
    os.chdir("..\Server")
os.getcwd()

# IRScout Python Modules
import scouting.output as out
from scouting.event import EventDal

# Use the Bokeh plotting package
import bokeh.charts as bc
import bokeh.charts.attributes as bca
os.environ["BOKEH_RESOURCES"] = "inline"
bc.output_notebook()

# Other useful modules
import pandas as pd
import sqlalchemy as sa

# Get a database connection
connection_string = ('postgresql://irs1318:irs1318'
                     '@localhost:5432/scouting')
engine = sa.create_engine(connection_string)
conn = engine.connect()

Loading BokehJS ...

### Set Event
* **Next, set `current_event` to the applicable FIRST API event code and run this cell.**

In [4]:
current_event = "turing"
EventDal.set_current_event(current_event)
EventDal.get_current_event()

u'turing'

### Export Excel Ranking File
* Run this cell to export the rankings spreadsheet.

In [5]:
fpath = out.get_Path("Report")
out.get_report(fpath)

### Get a Dataframe Containing all Measures

In [6]:
# Get Measures Dataframe
sql_measures = sa.text(
    "SELECT events.name AS event, matches.name AS match, levels.name AS level, dates.name AS date, "
    "teams.name AS team, alliances.name AS alliance, "
    "stations.name AS station, phases.name AS phase, actors.name AS actor, tasks.name AS task, "
    "measures.capability AS capability, measures.successes as successes, measures.attempts AS attempts, "
    "measures.cycle_times as cycle_times, measuretypes.name AS measuretype, reasons.name AS reason, "
    "attempts.name AS attempt "
    "FROM ((((((((((((measures FULL OUTER JOIN teams ON measures.team_id=teams.id) "
    "LEFT JOIN events ON measures.event_id=events.id) "
    "LEFT JOIN matches ON measures.match_id=matches.id) "
    "LEFT JOIN levels ON measures.level_id=levels.id) "
    "LEFT JOIN dates ON measures.date_id=dates.id) "
    "LEFT JOIN alliances ON measures.alliance_id=alliances.id) "
    "LEFT JOIN stations ON measures.station_id=stations.id) "
    "LEFT JOIN phases ON measures.phase_id=phases.id) "
    "LEFT JOIN actors ON measures.actor_id=actors.id) "
    "LEFT JOIN tasks ON measures.task_id=tasks.id) "
    "LEFT JOIN reasons ON measures.reason_id=reasons.id) "
    "LEFT JOIN measuretypes ON measures.measuretype_id=measuretypes.id) "
    "LEFT JOIN attempts ON measures.attempt_id=attempts.id "
    "WHERE events.name='" + current_event + "';")
measures = pd.read_sql(sql_measures, conn)
measures.query("task == 'placeGear'").query("phase == 'auto'").query("team == '1318'")
#measures.query("task == 'climbRope' and phase == 'finish'")

,event,match,level,date,team,alliance,station,phase,actor,task,capability,successes,attempts,cycle_times,measuretype,reason,attempt
98,turing,007-q,qual,2017-04-20T09:18:00,1318,blue,1,auto,robot,placeGear,0,1,1,0,boolean,na,summary
818,turing,019-q,qual,2017-04-20T10:54:00,1318,blue,3,auto,robot,placeGear,0,1,1,0,boolean,na,summary
2137,turing,043-q,qual,2017-04-20T16:08:00,1318,red,1,auto,robot,placeGear,0,1,1,0,boolean,na,summary
2492,turing,052-q,qual,2017-04-21T08:24:00,1318,blue,1,auto,robot,placeGear,0,0,1,0,boolean,na,summary
3020,turing,061-q,qual,2017-04-21T09:36:00,1318,blue,2,auto,robot,placeGear,0,1,1,0,boolean,na,summary
3708,turing,079-q,qual,2017-04-21T13:56:00,1318,red,2,auto,robot,placeGear,0,1,1,0,boolean,na,summary
4804,turing,100-q,qual,2017-04-21T16:44:00,1318,red,3,auto,robot,placeGear,0,1,1,0,boolean,na,summary
4934,turing,108-q,qual,2017-04-21T17:48:00,1318,red,2,auto,robot,placeGear,0,0,1,0,boolean,na,summary


In [7]:
    num_matches = 10
    evt = "turing"
    
    select_sum = sa.text(
        "with current AS (SELECT s.event as event, s.match, date from schedules sched, "
        "status s WHERE sched.event = s.event "
        "AND sched.match = s.match limit 1 ), "

        "recent_matches as ( SELECT * FROM ( "
        "SELECT row_number() over (partition by team order by sched.date desc) as r, "
        " sched.* from schedules sched, current c WHERE sched.event = c.event and sched.date <= c.date )"
        " row_schedule WHERE row_schedule.r <= " + str(
            num_matches) + " ORDER by team, date desc), "

        "team_match_count as ( "
        "select team, count(team) as team_matches from recent_matches group by team"
           ") "

           "SELECT teams.name AS team, phases.name AS phase, tasks.name AS task, actors.name AS actor, "
           "MAX(team_match_count.team_matches) AS matches, "
           "SUM(successes) AS sum_successes, SUM(attempts) AS sum_attempts, AVG(cycle_times) "
           "FROM ((((teams FULL OUTER JOIN measures ON teams.id=measures.team_id) "
           "LEFT JOIN tasks ON tasks.id = measures.task_id) "
           "LEFT JOIN phases ON phases.id = measures.phase_id) "
           "LEFT JOIN events ON events.id = measures.event_id) "
           "LEFT JOIN actors ON actors.id = measures.actor_id "
           "LEFT JOIN matches ON matches.id = measures.match_id "
           "LEFT JOIN team_match_count ON team_match_count.team = teams.name "
           "RIGHT JOIN recent_matches ON recent_matches.match = matches.name AND team_match_count.team = teams.name "
           "AND recent_matches.team = team_match_count.team "
           "WHERE events.name = '"+evt+"' AND actors.name<> 'alliance' "
           "GROUP BY teams.name, tasks.name, phases.name, actors.name "
           "ORDER BY teams.name, phases.name, tasks.name, actors.name;")
    df = pd.read_sql(select_sum, conn)

In [8]:
    num_matches = 12
    evt = "turing"
    
    select_recent_matches = sa.text(
        "with current AS (SELECT s.event, s.match, date from schedules sched, "
        "status s WHERE sched.event = s.event "
        "AND sched.match = s.match limit 1 ) "

 
        "SELECT * FROM ( "
        "SELECT row_number() over (partition by team order by sched.date desc) as r, "
        " sched.* from schedules sched, current c WHERE sched.event = c.event and sched.date <= c.date )"
        " row_schedule WHERE row_schedule.r <= " + str(
            num_matches) + " ORDER by team, date desc")
    df = pd.read_sql(select_recent_matches, conn)
    df

,r,id,date,event,level,match,alliance,team,station
0,1,18895,2017-04-21T18:20:00,turing,qual,112-q,red,108,1
1,2,18776,2017-04-21T15:40:00,turing,qual,092-q,red,108,2
2,3,18700,2017-04-21T13:56:00,turing,qual,079-q,blue,108,1
3,4,18671,2017-04-21T13:16:00,turing,qual,074-q,blue,108,2
4,5,18567,2017-04-21T09:04:00,turing,qual,057-q,red,108,3
5,6,18534,2017-04-21T08:16:00,turing,qual,051-q,blue,108,3
6,7,18442,2017-04-20T15:12:00,turing,qual,036-q,blue,108,1
7,8,18374,2017-04-20T11:42:00,turing,qual,025-q,red,108,2
8,9,18331,2017-04-20T10:46:00,turing,qual,018-q,red,108,1
9,10,18251,2017-04-20T08:54:00,turing,qual,004-q,blue,108,2


In [9]:
num_matches = df[['team', 'level']].groupby('team').count()
num_matches.columns = ['matches']
num_matches

,matches
team,
108,10
1165,10
1287,10
1296,10
1318,10
1323,10
1523,10
1540,10
1595,10


In [10]:
    num_matches = 10
    evt = "turing"
    
    select_current = sa.text(
        "SELECT s.event, s.match, date from schedules sched, "
        "status s WHERE sched.event = s.event "
        "AND sched.match = s.match limit 1")
    df = pd.read_sql(select_current, conn)
    df.head(100)

,event,match,date
0,turing,112-q,2017-04-21T18:20:00


### Create and Save Gears Chart
* This cell creates the gears chart and saves it to Server/web/data as a static, interactive html file.

**TODO:** Add an event code and timestamp to gears chart file name.

In [11]:
# Create summary data frame for placing gears
gears_df = measures.query("task == 'placeGear'").query("phase == 'auto' or phase == 'teleop'").groupby('team').sum()
gears_df = gears_df.sort_values('successes', ascending = False)

# Create gears chart in Bokeh
gears_chart = bc.Bar(gears_df, values = 'successes', label = bca.cat(sort = False), legend = None, color = 'purple',
                    title = 'Total Gears Placed', ylabel = "Gears Placed")
bc.output_file('web/data/gears_chart.html', mode = 'inline')
bc.show(gears_chart)

In [12]:
# Create summary data frame for placing gears
gears_df = measures.query("task == 'climbRope'").query("phase == 'finish'").groupby('team').count()
gears_df = gears_df.sort_values('successes', ascending = False)

# Create gears chart in Bokeh
climb_chart = bc.Bar(gears_df, values = 'successes', label = bca.cat(sort = False), legend = None, color = 'purple',
                    title = 'Total Times Climbed', ylabel = "Times Climbed")
bc.output_file('web/data/climb_chart.html', mode = 'inline')
bc.show(climb_chart)

INFO:bokeh.core.state:Session output file 'web/data/climb_chart.html' already exists, will be overwritten.


In [14]:
# Create summary data frame for shooting fuel
fuel_df = measures.query("task == 'shootHighBoiler'").query("phase == 'auto' or phase == 'teleop'").groupby('team').sum()
fuel_df = fuel_df.sort_values('successes', ascending = False)

# Create gears chart in Bokeh
fuel_chart = bc.Bar(fuel_df, values = 'successes', label = bca.cat(sort = False), legend = None, color = 'purple',
                    title = 'Total Fuel Shot', ylabel = "Number of Shots", responsive = True)
fuel_chart.title.text_color = "navy"
bc.output_file('web/data/fuel_chart.html', mode = 'inline')
bc.show(fuel_chart)

INFO:bokeh.core.state:Session output file 'web/data/fuel_chart.html' already exists, will be overwritten.


In [ ]:
# Create summary data frame for placing gears
gears_df = measures.query("task == 'placeGear'").query("phase == 'auto' or phase == 'teleop'")
gears_df = gears_df.sort_values('successes', ascending = False)

bc.defaults.height = 300
bc.defaults.width = 1200

# Create gears chart in Bokeh
# gears_chart = bc.BoxPlot(gears_df, values = 'successes', label = 'team', legend = None, color = 'purple',
#                     title = 'Total Gears Placed', ylabel = "Gears Placed")
gears_chart2 = bc.BoxPlot(gears_df, values = 'successes', label = 'team', legend = None, color = ['red', 'blue'])
bc.show(gears_chart2)